In [1]:
from pyspark import SparkContext
import math

## Configuração inicial

In [2]:
sc = SparkContext(appName="Teste")

In [3]:
rdd = sc.sequenceFile("pages/part-00000")

In [5]:
#total de documentos na base
total_docs = rdd.count()

#### Funções de tratamento

In [6]:
#Quantas ocorrencias da palavra em todos documentos
def conta_palavras(item):
    texto = item[1]
    palavras = texto.strip().split()
    return [(palavra.lower(),1) for palavra in palavras]

In [7]:
#Conta quantos docs cada palavra aparece
def conta_docs(item):
    texto = item[1]
    palavras = texto.strip().split()
    return [(palavra.lower(),1) for palavra in set(palavras)]
rdd_docs_word = rdd.flatMap(conta_docs).reduceByKey(lambda x,y: x + y).cache()

In [8]:
#Filtra palavras que aparecem em certa quantidade de docs
doc_freq_min = 10
doc_freq_max = 0.7 * total_docs
def filtra(item):
    contagem = item[1]
    return (contagem < doc_freq_max) and (contagem > doc_freq_min)

#RDD com quantos docs cada palavra aparece considerando intervalo limite de 5 a 0.7*total_documentos
rdd_docs_filtrado = rdd_docs_word.filter(filtra)

#### Análise do vocabulário comum

In [9]:
#Calcula idf de cada palavra filtrada
def computa_idf(item):
    palavra, contagem = item
    idf = math.log10(total_docs / contagem)
    return (palavra, idf)
rdd_idf = rdd_docs_filtrado.map(computa_idf)
rdd_idf.take(10)

[('pons', 3.4117759853793284),
 ('r$', 0.5932673172690647),
 ('//}', 3.3274550996792924),
 ('à', 0.32189609502544525),
 ('e-mail', 0.43047524320596725),
 ('bom', 0.8289013117853762),
 ('joaquim', 1.6987657237630354),
 ('cantores', 2.5086859983873846),
 ('região', 0.881576287176246),
 ('*', 0.864528870032381)]

#### Análise do vocabulário específico

In [10]:
#Filtrar RDDs para selecionar conjuntos com cada palavra definida.
rdd_oreo = rdd.filter(lambda x: "oreo" in x[1]) 
rdd_negresco = rdd.filter(lambda x: "negresco" in x[1]) 

In [11]:
#Cálculo da frequência para os RDDs dos dois conjuntos.
rdd_freq_oreo = rdd_oreo.flatMap(conta_palavras).reduceByKey(lambda x,y: x + y).map(lambda x: (x[0], math.log10(1 + x[1]))).cache()
rdd_freq_negresco = rdd_negresco.flatMap(conta_palavras).reduceByKey(lambda x,y: x + y).map(lambda x: (x[0], math.log10(1 + x[1]))).cache()

In [12]:
#Intersecção dos dois conjuntos de frequência.
rdd_inter = rdd_freq_oreo.intersection(rdd_freq_negresco)

##### Tabela com 100 palavras mais relevantes onde os dois itens aparecem conjuntamente

In [13]:
#Calcula 100 palavras mais relevantes em conjunto às palavras escolhidas
rdd_all = rdd_inter.join(rdd_idf).map(lambda x: (x[0], x[1][0]*x[1][1])).takeOrdered(1, key=lambda x: -x[1])
rdd_all.saveAsTextFile("pages/rdd_all.txt")

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 6 in stage 6.0 failed 1 times, most recent failure: Lost task 6.0 in stage 6.0 (TID 48, LAPTOP-EFUL12DG, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\Users\AndreWeber\AppData\Local\Programs\Python\Python38\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 605, in main
  File "C:\Users\AndreWeber\AppData\Local\Programs\Python\Python38\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 595, in process
  File "C:\Users\AndreWeber\AppData\Local\Programs\Python\Python38\lib\site-packages\pyspark\rdd.py", line 2596, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "C:\Users\AndreWeber\AppData\Local\Programs\Python\Python38\lib\site-packages\pyspark\rdd.py", line 425, in func
    ret

##### Tabela com 100 palavras mais relevantes sem a presença de "negresco"

In [15]:
#Calcula 100 palavras mais relevantes com apenas do conjunto Oreo
rdd_freq_oreo.subtractByKey(rdd_freq_negresco).join(rdd_idf).map(lambda x: (x[0], x[1][0]*x[1][1])).takeOrdered(100, key=lambda x: -x[1])

[('riachuelo', 8.378763843447578),
 ('soil', 7.882439711900808),
 ('teens', 6.914087807944809),
 ('great.', 6.683526884745895),
 ('i’m', 6.587384961277829),
 ('responda', 6.58399381824705),
 ('cool.', 6.554107335323577),
 ('forward', 6.48867612719286),
 ('rchlo', 6.384720996588554),
 ('thank', 6.328807730846682),
 ('i’ll', 6.319928024771057),
 ("levi's", 6.313044381390977),
 ('appreciate', 6.310778353563317),
 ('cachecol,', 6.3100349048863285),
 ('really', 6.30764695153316),
 ('article.', 6.29904898393505),
 ('you!', 6.264579233323951),
 ('more.', 6.2321536658164005),
 ('nfl', 6.208421463312811),
 ('suéter', 6.196256862404943),
 ('grow', 6.180697312642146),
 ('writing.', 6.139480882653706),
 ('education,', 6.129436513946753),
 ('bahls', 6.091145457070974),
 ('jerseys', 6.085719813604868),
 ('cells', 6.064057145561836),
 ('enjoyed', 6.047435310109896),
 ('loans', 5.993716764971986),
 ('{{item.name}}', 5.991479887551308),
 ('íntima', 5.990699037868668),
 ('this.', 5.988677304928174),
 ('

##### Tabela com 100 palavras mais relevantes sem a presença de "oreo"

In [19]:
#Calcula 100 palavras mais relevantes com apenas do conjunto Negresco
rdd_freq_negresco.subtractByKey(rdd_freq_oreo).join(rdd_idf).map(lambda x: (x[0], x[1][0]*x[1][1])).takeOrdered(100, key=lambda x: -x[1])

[('mármore', 5.833777920042529),
 ('juliana,', 5.772869418743419),
 ('granito', 5.6992696159764415),
 ('manteiga,', 5.158187549175862),
 ('mármores', 4.513494469386839),
 ('lavatório', 4.4543124815496755),
 ('juliana!', 4.280390686285018),
 ('granitos', 4.15973029572628),
 ('manteiga.', 4.145702536465033),
 ('confeiteiro', 4.14284416144329),
 ('margarina', 3.836646023263132),
 ('geladeira,', 3.6866352216031464),
 ('soleira', 3.602200958617797),
 ('açucar', 3.4766460532793557),
 ('receita,', 3.382433644093701),
 ('merengue', 3.3195451504692186),
 ('corante', 3.311231520847154),
 ('cotar', 3.2634298068495045),
 ('confeitar', 3.151696965500766),
 ('testei', 3.073622004679498),
 ('chocolate,', 3.0406377649871903),
 ('juliana.', 3.028768783613926),
 ('travertino', 3.0049913826856485),
 ('hahahaha.', 2.939861798913858),
 ('marmoraria', 2.864255767058688),
 ('sal,', 2.847771453817274),
 ('açúcar,', 2.8397800759331178),
 ('balcão', 2.787945755397189),
 ('","', 2.7585634716039844),
 ('m2', 2.71